In [1]:
import os
from os.path import join
import urllib.request #https://stackoverflow.com/questions/17960942/attributeerror-module-object-has-no-attribute-urlretrieve
from html_parser import retrieve_info_for_galaxy

In [2]:
#constants:
PATH_TO_GALAXY_INFO_FILE = "..\\table_info\\size_of_galaxies\\Table3_part3.txt"
DOWNLOAD_PATH = "..\\galaxies\\table3_part3"

In [3]:
#helper functions:
def get_names_and_pixel_size(file_path, delimeter=":"):
    name_to_pixel_dict = dict()
    with open(file_path,'r') as f:
        for each_line in f.readlines():
            if not each_line.startswith('#') and delimeter in each_line:
                name = each_line.split(delimeter)[0].strip()
                pixel_size = each_line.split(delimeter)[1].strip()
                name_to_pixel_dict[name] = pixel_size
    return name_to_pixel_dict

def write_to_file(to_write,folder,file_name):
    check_if_folder_exists_and_if_not_make_folder(folder)
    path = join(folder,file_name)
    
    with open(path, 'a') as out:
        out.write(to_write + '\n')
    out.close()

def check_if_folder_exists_and_if_not_make_folder(folder):
    '''create folder if does not exist'''
    if not os.path.exists(folder):
        os.makedirs(folder)
        
def download_fits_file(url_to_fit,name,band,is_mask=False):
    folder = join(DOWNLOAD_PATH,name)
    check_if_folder_exists_and_if_not_make_folder(folder)

    file_name_ending = "_mask" if is_mask else ""

    file_name = "{}_{}{}.fits".format(name,band,file_name_ending)
    path_to_save = join(folder,file_name)
    urllib.request.urlretrieve(url_to_fit,path_to_save)

def download_color(url_to_color, name):
    folder = join(DOWNLOAD_PATH,name)

    file_name = "{}_color.jfif".format(name)
    path_to_save = join(folder,file_name)
    urllib.request.urlretrieve(url_to_color,path_to_save)

In [4]:
#primary function:
def run_on_galaxy(name,pixel_size=240):
    (url, ra_dec_string, arcsec_size_string, color_image_link, fits_cutout_link_dict, fits_cutout_mask_link_dict) = retrieve_info_for_galaxy(name,pixel_size)
    folder = join(DOWNLOAD_PATH,name)
    
    #print(folder)
    
    if url != "":
        write_to_file(url,folder,"{}_link.txt".format(name))
    if ra_dec_string != "":
        write_to_file(ra_dec_string,folder,"{}_ra_dec.txt".format(name))
    if arcsec_size_string != "":
        size_string = "{} pixels {} arcsec".format(pixel_size,arcsec_size_string)
        write_to_file(size_string,folder,"{}_size.txt".format(name))
    if len(fits_cutout_link_dict) != 0:
        for each_band in fits_cutout_link_dict:
            url_to_fit = fits_cutout_link_dict[each_band]
            download_fits_file(url_to_fit,name,each_band,is_mask=False)
    if len(fits_cutout_link_dict) != 0:
        for each_band in fits_cutout_mask_link_dict:
            url_to_fit = fits_cutout_mask_link_dict[each_band]
            download_fits_file(url_to_fit,name,each_band,is_mask=True)
    if color_image_link != "":
        download_color(color_image_link, name)

In [5]:
#run with galaxy:

gals = get_names_and_pixel_size(PATH_TO_GALAXY_INFO_FILE)
total_gals = len(gals); current = 1



for each_gal in gals:
    the_size = gals[each_gal]
    
    print("{}: {} (#{} of {})".format(each_gal, the_size, current, total_gals))
    
    try:
        run_on_galaxy(each_gal,the_size)
        #break
    except Exception as e:
        print("Error running on galaxy: {}".format(each_gal))
        print(e)
        
    current += 1
        

    

NGC5971: 400 (#1 of 130)
NGC6004: 500 (#2 of 130)
NGC6032: 400 (#3 of 130)
NGC6060: 600 (#4 of 130)
NGC6063: 400 (#5 of 130)
NGC6154: 240 (#6 of 130)
NGC6155: 400 (#7 of 130)
NGC6301: 600 (#8 of 130)
NGC6478: 400 (#9 of 130)
NGC6497: 400 (#10 of 130)
NGC6509: 600 (#11 of 130)
NGC6814: 600 (#12 of 130)
NGC6941: 400 (#13 of 130)
NGC7047: 400 (#14 of 130)
NGC7217: 400 (#15 of 130)
NGC7321: 500 (#16 of 130)
NGC7489: 500 (#17 of 130)
NGC7549: 600 (#18 of 130)
NGC7653: 600 (#19 of 130)
NGC7664: 300 (#20 of 130)
NGC7691: 600 (#21 of 130)
NGC7716: 600 (#22 of 130)
NGC7723: 800 (#23 of 130)
NGC7741: 1000 (#24 of 130)
NGC7743: 600 (#25 of 130)
NGC7819: 600 (#26 of 130)
NGC7824: 600 (#27 of 130)
PGC02162: 400 (#28 of 130)
PGC03512: 240 (#29 of 130)
PGC05673: 500 (#30 of 130)
PGC06855: 300 (#31 of 130)
PGC07826: 300 (#32 of 130)
PGC08941: 300 (#33 of 130)
PGC14564: 240 (#34 of 130)
PGC15531: 400 (#35 of 130)
PGC16274: 240 (#36 of 130)
PGC19767: 400 (#37 of 130)
PGC20938: 240 (#38 of 130)
PGC23333: